# Phần 1: Code và câu trả lời cho "Thay đổi cấu trúc ANN".

In [2]:
# Cài đặt thư viện
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Đặt seed để kết quả ổn định hơn (tùy chọn)
torch.manual_seed(42)
np.random.seed(42)

In [3]:
# Tạo điểm cho lớp 0 (vòng tròn)
def generate_class_0(n_samples):
    u = np.random.uniform(0, 1, n_samples)
    theta = np.random.uniform(0, 2 * np.pi, n_samples)
    r = np.sqrt(u)  # Bán kính nhỏ
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    return np.column_stack((x, y))

# Tạo điểm cho lớp 1 (vành đai)
def generate_class_1(n_samples):
    u = np.random.uniform(0, 1, n_samples)
    theta = np.random.uniform(0, 2 * np.pi, n_samples)
    r = np.sqrt(3 * u + 1)  # Bán kính lớn hơn
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    return np.column_stack((x, y))

# Tạo dữ liệu
X_class0 = generate_class_0(100)
X_class1 = generate_class_1(200)
y_class0 = np.zeros(100)  # Nhãn 0
y_class1 = np.ones(200)   # Nhãn 1

# Kết hợp dữ liệu
X = np.vstack((X_class0, X_class1))
y = np.hstack((y_class0, y_class1))

# Chia dữ liệu
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Chuyển sang tensor PyTorch
X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train).unsqueeze(1)  # Thêm chiều cho nhãn
X_test = torch.FloatTensor(X_test)
y_test = torch.FloatTensor(y_test).unsqueeze(1)

1. Tăng số nút trong lớp ẩn:

In [4]:
# Xây dựng mô hình ANN 8 nút ẩn
class ANN(nn.Module):
    def __init__(self):
        super(ANN, self).__init__()
        self.layer1 = nn.Linear(2, 8)  
        self.relu = nn.ReLU()          # Công tắc ReLU
        self.layer2 = nn.Linear(8, 1)  
        self.sigmoid = nn.Sigmoid()    # Xác suất 0-1

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.sigmoid(x)
        return x

# Khởi tạo mô hình
model = ANN()

# Định nghĩa mất mát và tối ưu hóa
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Huấn luyện
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()       # Xóa gradient cũ
    outputs = model(X_train)    # Dự đoán
    loss = criterion(outputs, y_train)  # Tính sai lầm
    loss.backward()             # Tìm cách sửa
    optimizer.step()            # Sửa trọng số
    if (epoch + 1) % 20 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], loss: {loss.item():.4f}")

Epoch [20/100], loss: 0.5965
Epoch [40/100], loss: 0.5621
Epoch [60/100], loss: 0.5313
Epoch [80/100], loss: 0.4818
Epoch [100/100], loss: 0.4139


In [5]:
# Kiểm tra
model.eval()
with torch.no_grad():
    y_pred = model(X_test)
    y_pred = (y_pred > 0.5).float()  # Chuyển thành 0 hoặc 1
    accuracy = (y_pred.eq(y_test).sum() / y_test.size(0)).item()
    print(f"Độ chính xác: {accuracy*100:.2f}%")

Độ chính xác: 70.00%


2. Thêm một lớp ẩn:

In [6]:
# Xây dựng mô hình ANN 8+6 nút ẩn
class ANN(nn.Module):
    def __init__(self):
        super(ANN, self).__init__()
        self.layer1 = nn.Linear(2, 8)  
        self.relu = nn.ReLU()          # Công tắc ReLU
        self.layer2 = nn.Linear(8, 6)  
        self.layer3 = nn.Linear(6, 1)
        self.sigmoid = nn.Sigmoid()    # Xác suất 0-1

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.sigmoid(x)
        return x

# Khởi tạo mô hình
model = ANN()

# Định nghĩa mất mát và tối ưu hóa
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Huấn luyện
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()       # Xóa gradient cũ
    outputs = model(X_train)    # Dự đoán
    loss = criterion(outputs, y_train)  # Tính sai lầm
    loss.backward()             # Tìm cách sửa
    optimizer.step()            # Sửa trọng số
    if (epoch + 1) % 20 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], loss: {loss.item():.4f}")

Epoch [20/100], loss: 0.5710
Epoch [40/100], loss: 0.4888
Epoch [60/100], loss: 0.3534
Epoch [80/100], loss: 0.2098
Epoch [100/100], loss: 0.1244


In [7]:
# Kiểm tra
model.eval()
with torch.no_grad():
    y_pred = model(X_test)
    y_pred = (y_pred > 0.5).float()  # Chuyển thành 0 hoặc 1
    accuracy = (y_pred.eq(y_test).sum() / y_test.size(0)).item()
    print(f"Độ chính xác: {accuracy*100:.2f}%")

Độ chính xác: 98.33%


## Nhận xét:
mô hình 4 nút có độ chính xác 83.33% loss 0.3316 => dự đoán ổn

mô hình 8 nút có độ chính xác 70% loss 0.4139 => dự đoán chưa tốt

mô hình 8+6 nút có độ chính xác 98.33% loss 0.1244 => dự đoán tốt nhất


# Phần 2: Thử nghiệm với hàm mất mát và tối ưu hóa

In [ ]:
# BCEWithLogitsLoss
class ANN(nn.Module):
    def __init__(self):
        super(ANN, self).__init__()
        self.layer1 = nn.Linear(2, 4)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(4, 1)

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        return x

model = ANN()

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 20 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], loss: {loss.item():.4f}")


Epoch [20/100], loss: 0.5945
Epoch [40/100], loss: 0.5576
Epoch [60/100], loss: 0.5081
Epoch [80/100], loss: 0.4485
Epoch [100/100], loss: 0.3808


In [25]:
model.eval()
with torch.no_grad():
    logits = model(X_test)
    y_pred = torch.sigmoid(logits)
    y_pred = (y_pred > 0.5).float()
    accuracy = (y_pred.eq(y_test).sum() / y_test.size(0)).item()
    print(f"Độ chính xác: {accuracy*100:.2f}%")

Độ chính xác: 76.67%


In [ ]:
# Thay Adam bằng SGD
class ANN(nn.Module):
    def __init__(self):
        super(ANN, self).__init__()
        self.layer1 = nn.Linear(2, 4)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(4, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.sigmoid(x)
        return x

model = ANN()

criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)


epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 20 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], loss: {loss.item():.4f}")


Epoch [20/100], loss: 0.6580


Epoch [40/100], loss: 0.6544
Epoch [60/100], loss: 0.6512
Epoch [80/100], loss: 0.6483
Epoch [100/100], loss: 0.6456


In [27]:
model.eval()
with torch.no_grad():
    logits = model(X_test)
    y_pred = torch.sigmoid(logits)
    y_pred = (y_pred > 0.5).float()
    accuracy = (y_pred.eq(y_test).sum() / y_test.size(0)).item()
    print(f"Độ chính xác: {accuracy*100:.2f}%")

Độ chính xác: 63.33%


## So sánh:
### BCEWithLogitsLoss vs BCELoss

Mất mát:
BCEWithLogitsLoss cho loss thấp hơn (0.3808 so với 0.6456), cho thấy mô hình hội tụ tốt hơn.

Độ chính xác:
Độ chính xác cao hơn (76.67% vs 63.33%).

Nguyên nhân:
BCEWithLogitsLoss kết hợp Sigmoid và Binary Cross-Entropy trong một hàm duy nhất, giúp ổn định số học và gradient, tránh hiện tượng gradient nhỏ khi đầu ra gần 0 hoặc 1, nên mô hình học hiệu quả hơn.

### SGD vs Adam

Tốc độ giảm loss:
Adam làm loss giảm nhanh hơn, trong khi SGD giảm chậm và không ổn định với cùng số epoch.

Độ chính xác:
SGD cho độ chính xác thấp hơn đáng kể (63.33%) so với Adam (76.67%).

Nguyên nhân:
Adam tự điều chỉnh learning rate cho từng tham số và có momentum, còn SGD dùng learning rate cố định nên cần nhiều epoch hoặc tinh chỉnh thêm để đạt kết quả tương đương.

=> BCEWithLogitsLoss kết hợp Adam cho hiệu quả học tốt và ổn định hơn so với BCELoss và SGD trong cùng điều kiện huấn luyện.

# Phần 3: Phân tích kết quả